In [18]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("Output_results.csv")
labelsdf = pd.read_csv("genelabels.csv")

df = df.rename(columns={"Unnamed: 0": "Reaction"})

In [25]:
labelsdf['essential_new'].value_counts()

0    895
1    404
Name: essential_new, dtype: int64

In [22]:
labelsdf['essential_new'] = np.where((labelsdf['fdr']<0.05) & (labelsdf['beta'] < 0), 1, 0)

In [27]:
labelsdf.drop(['essential'], axis = 1)

,Gene.Name,Presence.,Bigg.ID.Name,beta,zscore,p.value,fdr,essential_new
0,A4GALT,True,53947_AT1,-0.109850,-0.92328,0.000249,0.001446,1
1,AACS,True,65985_AT1,0.160670,1.25820,0.090668,0.190420,0
2,AADAC,True,13_AT1,-0.023140,-0.25472,0.074658,0.163790,0
3,AADAT,True,51166_AT1,0.059995,0.63440,0.931650,0.958120,0
4,AASS,True,10157_AT1,-0.021270,-0.16418,0.082372,0.176850,0
...,...,...,...,...,...,...,...,...
1294,UROS,True,7390_AT1,-0.126860,-1.77140,0.000000,0.000000,1
1295,WWOX,True,51741_AT1,-0.072429,-0.95132,0.002572,0.010368,1
1296,XDH,True,7498_AT1,0.030918,0.48768,0.598590,0.738210,0
1297,XYLB,True,9942_AT1,-0.024406,-0.42204,0.071008,0.158080,0


In [29]:
#Splitting by "upper bound" and "lower bound"

tophalf = df.iloc[0:10600]
tophalf['Reaction'] = tophalf['Reaction']+'  (Upper)'

bottomhalf = df.iloc[10600:21200]
bottomhalf['Reaction'] = bottomhalf['Reaction']+'  (Lower)'

df = pd.concat([tophalf, bottomhalf], axis=0)

df = df.set_index('Reaction')

essentialdf = df.T
essentialdf = essentialdf.reindex(sorted(essentialdf.columns), axis=1)


KeyError: 'Reaction'

In [30]:
reactionlist = essentialdf.index.tolist()
newlist = []
for x in range(len(reactionlist)):
    newlist.append("N/A")

In [31]:
#Dictionary of GeneName as key, essential(0/1) as value
newdict = {}
IDnamelist = labelsdf['Bigg.ID.Name'].tolist()
Essentiallist =  labelsdf['essential_new'].tolist()
newdict = {IDnamelist[i]: Essentiallist[i] for i in range(len(IDnamelist))} 

listy = []
for x in range(len(reactionlist)):
    listy.append("N/A")
    
for i in range(len(essentialdf)):
    for key, value in newdict.items():
        if(essentialdf.index[i] == key):
            listy[i] = value
essentialdf['essential_new'] = listy  




In [32]:
#See which Genes have null essentiality values. These will be deleted.

tobedeleted = essentialdf[essentialdf['essential_new'] == "N/A"]
indices = tobedeleted.index.tolist()
essentialdf = essentialdf.drop(index = indices, axis = 0)

In [33]:
#Identify which features have zero variance

VarZero = (essentialdf.var() == 0)
VarianceZero = VarZero[VarZero == True]
listtodelete = VarianceZero.index.tolist()
essentialdf = essentialdf.drop(listtodelete, axis = 1)
genelist = essentialdf.index.tolist()

In [35]:
#Standardization of features
from sklearn.preprocessing import StandardScaler
names = essentialdf.columns
labels = essentialdf['essential_new'].tolist()
names = names.drop(['essential_new'])
scaler = StandardScaler()
scaleddf = scaler.fit_transform(essentialdf.drop(['essential_new'], axis=1))
finaldf = pd.DataFrame(scaleddf, columns=names)

#Resetting the Index as the GEne Name
finaldf['essential_new'] = labels
finaldf['GeneName'] = genelist
finaldf = finaldf.set_index(finaldf['GeneName'])
finaldf = finaldf.drop(['GeneName'], axis = 1)

In [36]:
finaldf.head()

Reaction,10FTHF5GLUtl (Lower),10FTHF5GLUtl (Upper),10FTHF5GLUtm (Lower),10FTHF5GLUtm (Upper),10FTHF6GLUtl (Lower),10FTHF6GLUtl (Upper),10FTHF6GLUtm (Lower),10FTHF6GLUtm (Upper),10FTHF7GLUtl (Lower),10FTHF7GLUtm (Lower),...,r2535m (Lower),r2535m (Upper),r2537 (Lower),r2537 (Upper),r2538 (Lower),r2538 (Upper),r2539 (Lower),r2539 (Upper),thr_mt (Lower),essential_new
GeneName,,,,,,,,,,,,,,,,,,,,,
53947_AT1,-0.094323,-0.061771,-0.051165,-0.004331,0.045383,0.078885,0.039759,0.053827,0.074091,0.036798,...,-0.018348,-0.801389,0.951153,-1.456913,1.805275,-1.456913,0.277695,-20.784610,0.690347,1
65985_AT1,-0.044243,-0.061771,-0.000986,-0.004331,-0.021091,0.078885,-0.026154,0.053827,0.074091,-0.023073,...,-0.018348,-0.801389,0.951153,-1.456913,1.805275,-1.456913,0.277695,-20.784610,0.690347,0
13_AT1,-0.044243,-0.061771,-0.000986,-0.004331,-0.021091,0.078885,-0.026154,0.053827,0.074091,-0.023073,...,-0.018348,-0.801389,0.951153,-1.456913,1.805275,-1.456913,0.277695,-20.784610,0.690347,0
51166_AT1,-3.985429,0.012277,-3.949980,0.059070,-5.521336,0.078885,-4.960144,6.236846,-3.251122,-3.825138,...,-0.018348,-0.801389,0.951153,-1.456913,1.805275,-1.456913,0.277695,0.048113,0.690347,0
10157_AT1,-0.058712,0.012277,-0.015483,-0.125263,-0.030128,0.078885,-0.035115,0.053827,0.089752,-0.031213,...,-0.018348,-0.801389,0.951153,-1.456913,1.805275,-1.456913,0.277695,0.048113,0.690347,0


In [37]:
finaldf.groupby('essential_new').mean()

Reaction,10FTHF5GLUtl (Lower),10FTHF5GLUtl (Upper),10FTHF5GLUtm (Lower),10FTHF5GLUtm (Upper),10FTHF6GLUtl (Lower),10FTHF6GLUtl (Upper),10FTHF6GLUtm (Lower),10FTHF6GLUtm (Upper),10FTHF7GLUtl (Lower),10FTHF7GLUtm (Lower),...,r2535 (Upper),r2535m (Lower),r2535m (Upper),r2537 (Lower),r2537 (Upper),r2538 (Lower),r2538 (Upper),r2539 (Lower),r2539 (Upper),thr_mt (Lower)
essential_new,,,,,,,,,,,,,,,,,,,,,
0,0.006042,-0.002153,0.023274,-0.019648,0.029837,0.013116,0.028126,0.054777,0.020666,0.018608,...,0.019588,-0.004468,0.002272,-0.015421,0.001486,-0.016191,0.001486,0.035001,0.001559,0.006347
1,-0.013386,0.004770,-0.051560,0.043527,-0.066099,-0.029056,-0.062308,-0.121350,-0.045783,-0.041223,...,-0.043394,0.009897,-0.005033,0.034163,-0.003292,0.035869,-0.003292,-0.077539,-0.003454,-0.014061


In [38]:
finaldf.groupby('essential_new').mean()

Reaction,10FTHF5GLUtl (Lower),10FTHF5GLUtl (Upper),10FTHF5GLUtm (Lower),10FTHF5GLUtm (Upper),10FTHF6GLUtl (Lower),10FTHF6GLUtl (Upper),10FTHF6GLUtm (Lower),10FTHF6GLUtm (Upper),10FTHF7GLUtl (Lower),10FTHF7GLUtm (Lower),...,r2535 (Upper),r2535m (Lower),r2535m (Upper),r2537 (Lower),r2537 (Upper),r2538 (Lower),r2538 (Upper),r2539 (Lower),r2539 (Upper),thr_mt (Lower)
essential_new,,,,,,,,,,,,,,,,,,,,,
0,0.006042,-0.002153,0.023274,-0.019648,0.029837,0.013116,0.028126,0.054777,0.020666,0.018608,...,0.019588,-0.004468,0.002272,-0.015421,0.001486,-0.016191,0.001486,0.035001,0.001559,0.006347
1,-0.013386,0.004770,-0.051560,0.043527,-0.066099,-0.029056,-0.062308,-0.121350,-0.045783,-0.041223,...,-0.043394,0.009897,-0.005033,0.034163,-0.003292,0.035869,-0.003292,-0.077539,-0.003454,-0.014061


In [39]:
finaldf.groupby('Essential')['10FTHF5GLUtl  (Lower)'].plot(legend=True)

KeyError: 'Essential'

In [41]:
finaldf

Reaction,10FTHF5GLUtl (Lower),10FTHF5GLUtl (Upper),10FTHF5GLUtm (Lower),10FTHF5GLUtm (Upper),10FTHF6GLUtl (Lower),10FTHF6GLUtl (Upper),10FTHF6GLUtm (Lower),10FTHF6GLUtm (Upper),10FTHF7GLUtl (Lower),10FTHF7GLUtm (Lower),...,r2535m (Lower),r2535m (Upper),r2537 (Lower),r2537 (Upper),r2538 (Lower),r2538 (Upper),r2539 (Lower),r2539 (Upper),thr_mt (Lower),essential_new
GeneName,,,,,,,,,,,,,,,,,,,,,
53947_AT1,-0.094323,-0.061771,-0.051165,-0.004331,0.045383,0.078885,0.039759,0.053827,0.074091,0.036798,...,-0.018348,-0.801389,0.951153,-1.456913,1.805275,-1.456913,0.277695,-20.784610,0.690347,1
65985_AT1,-0.044243,-0.061771,-0.000986,-0.004331,-0.021091,0.078885,-0.026154,0.053827,0.074091,-0.023073,...,-0.018348,-0.801389,0.951153,-1.456913,1.805275,-1.456913,0.277695,-20.784610,0.690347,0
13_AT1,-0.044243,-0.061771,-0.000986,-0.004331,-0.021091,0.078885,-0.026154,0.053827,0.074091,-0.023073,...,-0.018348,-0.801389,0.951153,-1.456913,1.805275,-1.456913,0.277695,-20.784610,0.690347,0
51166_AT1,-3.985429,0.012277,-3.949980,0.059070,-5.521336,0.078885,-4.960144,6.236846,-3.251122,-3.825138,...,-0.018348,-0.801389,0.951153,-1.456913,1.805275,-1.456913,0.277695,0.048113,0.690347,0
10157_AT1,-0.058712,0.012277,-0.015483,-0.125263,-0.030128,0.078885,-0.035115,0.053827,0.089752,-0.031213,...,-0.018348,-0.801389,0.951153,-1.456913,1.805275,-1.456913,0.277695,0.048113,0.690347,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390_AT1,-0.094323,0.012277,-0.051165,0.059070,-0.052372,0.078885,-0.057172,0.053827,0.011450,-0.051247,...,-0.018348,1.247833,-1.051356,0.686383,-0.553932,0.686383,0.277695,0.048113,0.138834,1
51741_AT1,-0.094323,0.012277,-0.051165,0.059070,-0.052372,0.078885,-0.057172,0.053827,0.011450,-0.051247,...,-0.018348,1.247833,-1.051356,0.686383,-0.553932,0.686383,0.277695,0.048113,0.138834,1
7498_AT1,-0.094323,0.012277,-0.051165,0.059070,-0.052372,0.078885,-0.057172,0.053827,0.011450,-0.051247,...,-0.018348,1.247833,-1.051356,0.686383,-0.553932,0.686383,0.277695,0.048113,0.138834,0


In [40]:
finaldf.to_csv('UpdatedEssentials.csv')

In [14]:
df

,498_AT1,506_AT1,5048_AT1,64241_AT1,131_AT1,9420_AT1,26279_AT1,7166_AT1,8876_AT1,53947_AT1,...,7384_AT1,7385_AT1,7386_AT1,7388_AT1,7389_AT1,7390_AT1,51741_AT1,7498_AT1,9942_AT1,284273_AT1
Reaction,,,,,,,,,,,,,,,,,,,,,
24_25DHVITD3tm (Upper),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25HVITD3t (Upper),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COAtl (Upper),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EX_5adtststerone_e (Upper),-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EX_5adtststerones_e (Upper),-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,-1.000000e+03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACMPGLUTTRsc (Lower),6.878054e-12,-2.842171e-14,-2.842171e-14,0.000000e+00,0.000000e+00,2.376701e-13,2.376701e-13,2.376701e-13,1.136868e-13,7.815970e-14,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
FVSCOAhc (Lower),6.878054e-12,-2.273737e-13,-2.273737e-13,6.821210e-13,6.821210e-13,2.376701e-13,2.376701e-13,2.376701e-13,1.136868e-13,7.815970e-14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MDZGLChr (Lower),1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
